# Script to make the injections json file

Some parameters are set, some are randomly generated

The distance is set by calculating how close it would be to have a full-band SNR of 2000

Note that the randomly generated values we use have a set number of significant figures when they are saved, as does the distance; this means that the SNR out may not be exactly 2000

## Imports / common variable definitions

In [1]:
import numpy
import json
import copy

import pycbc
import pycbc.psd
import pycbc.filter
import pycbc.waveform

# Common things needed for the waveform generation (when calculating SNR)
f_nyquist = 0.1
t_length = 86400 * 30
delta_f = 1. / t_length
delta_t = 0.25

channels = ['A','E','T']


/home/gareth/miniconda3/envs/env_lisa_ew/lib/python3.10/site-packages/pycbc/types/array.py:36: UserWarning: Wswiglal-redir-stdio:

SWIGLAL standard output/error redirection is enabled in IPython.
This may lead to performance penalties. To disable locally, use:

with lal.no_swig_redirect_standard_output_error():
    ...

To disable globally, use:

lal.swig_redirect_standard_output_error(False)

Note however that this will likely lead to error messages from
LAL functions being either misdirected or lost when called from
Jupyter notebooks.

To suppress this warning, use:

import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
import lal

  import lal as _lal


No CuPy
No CuPy or GPU PhenomHM module.
No CuPy or GPU response available.
No CuPy or GPU interpolation available.


## Set up chosen injection parameters

These injection parameters have been chosen to make certain scientific points

In [2]:
injections = {}
for i in range(5):
    injections[i] = {}

# Set injection 0 parameters:
injections[0]['mass1'] = 1e6
injections[0]['mass2'] = 1e6
injections[0]['spin1z'] = 0
injections[0]['spin2z'] = 0

# Set injection 1 parameters:
injections[1]['mass1'] = 2e6
injections[1]['mass2'] = 5e5
injections[1]['spin1z'] = 0
injections[1]['spin2z'] = 0

# Set injection 2 parameters:
injections[2]['mass1'] = 1e6
injections[2]['mass2'] = 7e5
injections[2]['spin1z'] = 0.4
injections[2]['spin2z'] = -0.3

# Set injection 3 parameters:
injections[3]['mass1'] = 2.5e6
injections[3]['mass2'] = 2.5e6
injections[3]['spin1z'] = 0.8
injections[3]['spin2z'] = 0.9

# Set injection 4 parameters:
injections[4]['mass1'] = 1e7
injections[4]['mass2'] = 1e7
injections[4]['spin1z'] = 0
injections[4]['spin2z'] = 0

# Add random times to end of data to represent up to hour wait time
injections[0]['additional_end_data'] = 1050
injections[1]['additional_end_data'] = 3400
injections[2]['additional_end_data'] = 100
injections[3]['additional_end_data'] = 3100
injections[4]['additional_end_data'] = 1700

# Include the distance - this is designed to give fullband SNR of 2000,
# but will be slightly different due to config errors
injections[0]['distance'] = 27658.011507544677
injections[1]['distance'] = 17396.850629145487
injections[2]['distance'] = 12042.46921545863
injections[3]['distance'] = 102915.54392849082
injections[4]['distance'] = 11259.17078067498

In [3]:
waveform_gen_common = {
    'ifos': ['LISA_A','LISA_E','LISA_T'],
    'approximant': 'BBHX_PhenomHM',
    'delta_f': delta_f,
    'f_lower': 1e-6,
    't_obs_start': t_length,
    't_offset': 0,
    'f_final': 0.1,
    'low_frequency_cutoff': 0.000001,
    'tdi': '1.5',
}

In [4]:
psd_file_fmat = '../PSD_Files/model_{channel}_TDI1_optimistic.txt.gz'
psds = {}
for channel in channels:
    if channel in ['A','E']:
        channel_long = 'AE'
    else:
        channel_long = channel
    psds[channel] = pycbc.psd.from_txt(
        psd_file_fmat.format(channel=channel_long),
        int(f_nyquist/delta_f),
        delta_f,
        delta_f,
        is_asd_file=False
    )

In [5]:
def get_snr_at_merger(inj):
    # Now we want to work out the SNR at merger - sigma is the SNR at 1Mpc
    wf = pycbc.waveform.get_fd_det_waveform(**inj)
    sigsq = 0
    for channel in channels:
        sigsq += pycbc.filter.sigma(
            wf[f'LISA_{channel}'],
            psds[channel],
            low_frequency_cutoff=inj['f_lower'],
            high_frequency_cutoff=inj['f_final'],
        ) ** 2

    return numpy.sqrt(sigsq)

In [6]:
for i in range(5):
    # For each injection, now randomly assign the external parameters:

    # Make this the same when repeated: set the seed
    numpy.random.seed(i * 1865)

    # longitude - uniform in 0, 2pi
    injections[i]['eclipticlongitude'] = numpy.random.uniform(0, 2 * numpy.pi)
    # latitude - uniform in sin latitude
    lat = numpy.arcsin(2 * numpy.random.uniform(0, 1) - 1)
    # round this to 2 significant figures:
    injections[i]['eclipticlatitude'] = lat
    # inclination - uniform in cos iota
    iota = numpy.arccos(numpy.random.uniform(0, 1))
    injections[i]['inclination'] = iota
    # polarisation - uniform in 0-2pi
    injections[i]['polarization'] = numpy.random.uniform(0, 2 * numpy.pi)
    # phase - uniform in 0-2pi
    injections[i]['coa_phase'] = numpy.random.uniform(0, 2 * numpy.pi)
    # Time of coalescence - uniform in the range
    # 1829952018 (2038-01-01T00:00:00) to 1987718418 (2043-01-01T00:00:00)
    injections[i]['tc'] = numpy.random.uniform(1829952018, 1987718418)

    # Now we want to work out the SNR at merger - sigma is the SNR at 1Mpc
    inj = copy.deepcopy(injections[i])
    # Add in the required definitions for the waveform generator:
    inj.update(waveform_gen_common)
    
    # Check that this has worked, the nw sigma should be 2000:
    sig = get_snr_at_merger(inj)
    print(f"Injection {i} full-signal SNR: {sig:.6f}")
    injections[i]['fullband_snr'] = sig

/home/gareth/miniconda3/envs/env_lisa_ew/lib/python3.10/site-packages/BBHX_Phenom.py:265: RuntimeWarning: Input 'f_lower' is lower than the value calculated from 't_obs_start'.
  warn(err_msg, RuntimeWarning)


Injection 0 full-signal SNR: 1975.470721


/home/gareth/miniconda3/envs/env_lisa_ew/lib/python3.10/site-packages/BBHX_Phenom.py:265: RuntimeWarning: Input 'f_lower' is lower than the value calculated from 't_obs_start'.
  warn(err_msg, RuntimeWarning)


Injection 1 full-signal SNR: 2124.626544
Injection 2 full-signal SNR: 2049.617596
Injection 3 full-signal SNR: 1990.494381
Injection 4 full-signal SNR: 2396.265255


In [7]:
with open('injections.json','w') as inj_json:
    json.dump(injections, inj_json)